Uge 20 + 21

Imports og klargøring af data

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

# Load og skaler data
data = load_breast_cancer()
X = data.data
y = data.target

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# RNN kræver 3D input: (samples, sequence_length, features)
X_seq = X_scaled.reshape((X_scaled.shape[0], 10, 3))  # 10 timesteps à 3 features

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_seq, y, test_size=0.2, random_state=42)

# Konverter til tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


Definer RNN-modller (LSTM og GRU)

In [5]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, rnn_type="LSTM"):
        super(RNNClassifier, self).__init__()
        self.rnn_type = rnn_type
        if rnn_type == "LSTM":
            self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        elif rnn_type == "GRU":
            self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        else:
            raise ValueError("Vælg enten 'LSTM' eller 'GRU'")
        
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]  # brug sidste tidssteg
        out = self.fc(out)
        return self.sigmoid(out)


Trænings- og evalueringsfunktion

In [6]:
def train_model(model, train_loader, epochs=20, lr=0.001):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            preds = (outputs > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"Test Accuracy: {correct / total:.2f}")


Træn og evaluer LSTM og GRU

In [10]:
for rnn_type in ["LSTM", "GRU"]:
    print(f"\nTræner {rnn_type} model:")
    model = RNNClassifier(input_size=3, hidden_size=32, rnn_type=rnn_type)
    train_model(model, train_loader, epochs=20)
    evaluate_model(model, test_loader)



Træner LSTM model:
Epoch 1, Loss: 19.5123
Epoch 2, Loss: 19.0794
Epoch 3, Loss: 17.7137
Epoch 4, Loss: 12.2621
Epoch 5, Loss: 8.9181
Epoch 6, Loss: 8.2776
Epoch 7, Loss: 7.5474
Epoch 8, Loss: 7.0457
Epoch 9, Loss: 7.5986
Epoch 10, Loss: 6.8994
Epoch 11, Loss: 6.3209
Epoch 12, Loss: 6.4415
Epoch 13, Loss: 6.2229
Epoch 14, Loss: 6.0635
Epoch 15, Loss: 6.1301
Epoch 16, Loss: 5.9285
Epoch 17, Loss: 6.0567
Epoch 18, Loss: 5.8915
Epoch 19, Loss: 5.6796
Epoch 20, Loss: 5.9673
Test Accuracy: 0.95

Træner GRU model:
Epoch 1, Loss: 19.5546
Epoch 2, Loss: 19.0743
Epoch 3, Loss: 18.3770
Epoch 4, Loss: 15.6546
Epoch 5, Loss: 9.1394
Epoch 6, Loss: 7.8356
Epoch 7, Loss: 7.3411
Epoch 8, Loss: 6.8411
Epoch 9, Loss: 6.3146
Epoch 10, Loss: 7.0504
Epoch 11, Loss: 6.3296
Epoch 12, Loss: 6.6261
Epoch 13, Loss: 6.6350
Epoch 14, Loss: 6.9358
Epoch 15, Loss: 6.2774
Epoch 16, Loss: 6.1204
Epoch 17, Loss: 5.7863
Epoch 18, Loss: 5.8007
Epoch 19, Loss: 5.6758
Epoch 20, Loss: 5.9897
Test Accuracy: 0.95


Ny patient test

In [15]:
new_patient = torch.tensor(X_test[0].reshape(1, 10, 3), dtype=torch.float32)
model.eval()
with torch.no_grad():
    output = model(new_patient)
    prediction = "Kræft" if output.item() > 0.5 else "Rask"
    print("Forudsigelse:", prediction)


Forudsigelse: Kræft
